<a href="https://colab.research.google.com/github/cedamusk/AI-N-ML/blob/main/Seismic_prediction_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import subprocess
import pkg_resources
from IPython.display import HTML, display
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def install_dependencies():
  print("Installing Python dependencies...")
  subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "tensorflow", "keras"])
  python_packages={
      'numpy':'1.23.5',
      'tensorflow':'2.15.0',
      'scipy':'1.11.4',
      'scikit-learn':'1.3.2',
      'pandas':'2.1.4',
      'matplotlib':'3.8.2',
      'plotly':'5.18.0',
      'ipywidgets':'8.1.1'
  }
  for package, version in python_packages.items():
    subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package}=={version}"])

  subprocess.check_call([sys.executable, "-m", "pip", "install", "tensorflow==2.13.0"])

  print("\nInstalling Node.js and npm...")
  subprocess.check_call(["apt-get", "update"])
  subprocess.check_call(["apt-get", "install", "-y", "nodejs", "npm"])

  print("\nInstalling Javascrpt dependencies...")
  subprocess.check_call(["npm", "init", "-y"])
  subprocess.check_call(["npm", "install", "react@17.0.2", "react-dom@17.0.2", "recharts@2.10.3"])



In [ ]:
!pip install tensorflow==2.15.0
!pip install keras

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from scipy import signal
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from google.colab import output
import json
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
REACT_COMPONENT="""
const SeismicDashboard=({data, predictions, metrics})=>{
  const timeSeriesData=data.map((value, index)=>({
    time:index,
    actual:value,
    predicted:predictions[index]||null
  }));
  const metricsData=[
    {name: 'Train MSE', value: metrics.train_mse},
    {name: 'Train MAE', value: metrics.train_mae},
    {name: 'Val MSE', value: metrics.val_mse},
    {name: 'Val MAE', value:metrics.val_mae}
  ];

  return(
    <div className="p-4 max-w-full">
      <div className="grid grid-cols-1 md:grid-cols-2 gap-4">
        {/*Seismic Waveform*/}
        <div className="bg-white p-4 rounded-lg shadow">
          <h3 className="text-lg font-semibold mb-4">Seismic Waveform</h3>
          <div className="h-64">
            <ResponsiveContainer width="100%" height="100%">
              <LineChart data={timeseriesData}>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis datakey="time" />
                <YAxis />
                <Tooltip />
                <Legend />
                <Line type="monotone" dataKey="actual" stroke="#8884d8" name="Actual" />
                <Line type="monotone" dataKey="predicted" stroke="#82ca9d" name="Predicted" />
              </LineChart>
            </ResponsiveContainer>
          </div>
        </div>

        {/*Training History*/}
        <div className="bg-white p-4 rounded-lg shadow">
          <h3 className="text-lg font-semibold mb-4">Training History</h3>
          <div className="h-64">
            <ResponsiveContainer width="100%" height="100%">
              <LineChart data={metrics.history}>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis dataKey="epoch" />
                <YAxis />
                <Tooltip />
                <Legend />
                <Line type="monotone" dataKey="loss" stroke="#8884d8" name="Training Loss"/>
                <Line type="monotone" dataKey="val_loss" stroke="#82ca9d" name="Validation Loss" />
              </LineChart>
            </ResponsiveContainer>
          </div>
        </div>

        {/*Performance Metrics*/}
        <div className="bg-white p-4 rounded-lg shadow">
          <h3 className="text-lg font-semibold mb-4">Performance Metrics</h3>
          <div className="h-64">
            <ResponsiveContainer width="100%" height="100%">
              <BarChart data={metricsData}>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis dataKey="name" />
                <YAxis />
                <Tooltip />
                <Bar dataKey="value" fill='#8884d8' />
              </BarChart>
            </ResponsiveContainer>
          </div>
        </div>

        {/*Power Spectral Density*/}
        <div className="bg-white p-4 rounded-lg shadow">
          <h3 className="text-lg font-semibold mb-4"> Power Spectral Density</h3>
          <div className="h-64">
            <ResponsiveContainer width="100%" height="100%">
              <LineChart data={data.psd}>
                <CartesianGrid strokeDasharray="3 3" />
                <XAxis dataKey="frequency" />
                <YAxis scale="log" />
                <Tooltip />
                <Line type="monotone" dataKey="power" stroke="#8884d8" dot={false} />
              </LineChart>
            </ResponsiveContainer>
          </div>
        </div>
      </div>
    <div>
  );
};
"""

In [ ]:
class EarthquakePredictionModel:
  def __init__(self, sequence_length=100, prediction_horizon=24):
    self.sequence_length=sequence_length
    self.prediction_horizon=prediction_horizon
    self.scaler=MinMaxScaler()
    self.model=None
    self.metrics_history=[]

  def generate_synthetic_data(self, num_samples=1000, sampling_rate=100):
    time=np.linspace(0, num_samples/sampling_rate, num_samples)
    data=np.zeros(num_samples)

    background=np.random.normal(0, 0.01, num_samples)

    for _ in range(np.random.randint(3, 8)):
      arrival_time=np.random.randint(0, num_samples)

      p_wave=0.5*np.exp(-0.1*np.abs(time*sampling_rate-arrival_time))*\
              np.sin(2*np.pi*10*(time*sampling_rate-arrival_time))

      s_wave=np.exp(-0.05*np.abs(time*sampling_rate-(arrival_time+50)))*\
              np.sin(2*np.pi*5*(time*sampling_rate-(arrival_time+50)))

      data += p_wave+s_wave

    data += background

    spike_locations=np.random.randint(0, num_samples, 5)
    data[spike_locations] += np.random.uniform(0.5, 2, 5)

    return data

  def calculate_psd(self, data, fs=100):
    frequencies, psd=signal.welch(data, fs, nperseg=min(256, len(data)))
    return frequencies, psd

  def prepare_sequences(self, data):
    sequences=[]
    targets=[]

    for i in range(len(data)-self.sequence_length-self.prediction_horizon):
      seq=data[i:(i+self.sequence_length)]
      _, psd=self.calculate_psd(seq)

      if len(psd) != self.sequence_length:
        psd=signal.resample(psd, self.sequence_length)

      combined_features=np.stack([seq, psd], axis=1)
      sequences.append(combined_features)
      targets.append(data[i+self.sequence_length +self.prediction_horizon])

    return np.array(sequences), np.array(targets)

  def build_model(self, input_shape):
    self.model=tf.keras.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=True,
                              input_shape=input_shape),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    optimizer =tf.keras.optimizers.Adam(learning_rate=0.001)
    self.model.compile(optimizer='adam', loss='mse',
                         metrics=['mae'])

    print("Model built successfully!")
    self.model.summary()

  def calculate_metrics(self, X, y, X_val, y_val):
    train_pred=self.model.predict(X)
    val_pred=self.model.predict(X_val)

    metrics={
        'train_mse': float(mean_squared_error(y, train_pred)),
        'train_mae': float(mean_absolute_error(y, train_pred)),
        'val_mse': float(mean_squared_error(y_val, val_pred)),
        'val_mae': float(mean_absolute_error(y_val, val_pred)),
        'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
      }

    self.metrics_history.append(metrics)
    return metrics

  def train(self, train_data, epochs=50, batch_size=32, validation_split=0.2):
    print("Preparing data for training...")

    scaled_data=self.scaler.fit_transform(train_data.reshape(-1, 1))

    X, y=self.prepare_sequences(scaled_data.flatten())

    split_idx=int(len(X)*(1-validation_split))
    X_train, X_val=X[:split_idx], X[split_idx:]
    y_train, y_val=y[:split_idx], y[split_idx:]

    print("\nStarting model training...")
    history=self.model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True
              ),
              tf.keras.callbacks.ProgbarLogger(count_mode='steps')
          ],
          verbose=1
      )

    print("\nCalculating performance metrics...")
    metrics=self.calculate_metrics(X_train, y_train, X_val, y_val)

    metrics['history']=[
        {
          'epoch':i+1,
          'loss': float(history.history['loss'][i]),
          'val_loss': float(history.history['val_loss'][i])
        }
        for i in range(len(history.history['loss']))
    ]

    return metrics

  def predict(self, sequence):
    if self.model is None:
      raise ValueError("Model not trained yet")

    scaled_sequence=self.scaler.transform(sequence.reshape(-1, 1))

    _, psd=self.calculate_psd(scaled_sequence.flatten())
    combined_features=np.concatenate([
          scaled_sequence,
          psd.reshape(-1,1)
      ])

    prediction=self.model.predict(combined_features.reshape(1, -1, 2))

    return self.scaler.inverse_transform(prediction.reshape(-1, 1))

  def visualize_in_colab(self, test_sequence, predictions, metrics):
    frequencies, psd_values=self.calculate_psd(test_sequence)
    psd_data=[{"frequency": float(f), "power":float(p)}
              for f, p in zip(frequencies, psd_values)]

    viz_data={
        "data": test_sequence.tolist(),
        "predictions": predictions.flatten().tolist(),
        "metrics": metrics,
        "psd": psd_data
      }

    html_content=f"""
    <!DOCTYPE html>
    <html>
    <head>
      <title>Earthquake Prediction Dashboard</title>
      <script src="https://cdnjs.cloudflare.com/ajax/libs/react/17.0.2/umd/react.production.min.js"></script>
      <script src="https://cdnjs.cloudflare.com/ajax/libs/react-dom/17.0.2/umd/react-dom.production.min.js"></script>
      <script src="https://cdnjs.cloudflare.com/ajax/libs/recharts/2.10.3/Recharts.min.js"></script>
      <link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css"rel="stylesheet">
    </head>
    <body>
        <div id="dashboard"></div>
        <script>
            const{{
                LineChart, BarChart, XAxis, YAxis, CartesianGrid, Tooltip,Legend, Line,
                Bar, ResponsiveConatiner
              }}=Recharts;

            {REACT_COMPONENT}

            const data={json.dumps(viz_data)};

            ReactDOM.render(
              React.createElement(SeismicDashboard, data),
              document.getElementById('dashboard')
              );
        </script>
    </body>
    </html>
    """

    display(HTML(html_content))


In [ ]:
def main():
  print("Starting Earthquake Prediction System...")

  try:
    install_dependencies()
    print("\nAll dependencies installed successfully!")
  except Exception as e:
    print(f"\n Error during installation: {str(e)}")
    return

  model =EarthquakePredictionModel()

  print("\nGenerating synthetic data...")
  synthetic_data=np.concatenate([
      model.generate_synthetic_data()
      for _ in tqdm(range(10))
  ])

  print("\nBuilding and training model...")
  input_shape=(model.sequence_length, 2)
  model.build_model(input_shape)
  metrics=model.train(synthetic_data, epochs=50)

  print("\nGenerating predictions...")
  test_sequence=model.generate_synthetic_data()
  prediction=model.predict(test_sequence)

  print("\nCreating interactive visualization...")
  model.visualize_in_colab(test_sequence, predictions, metrics)

  return model, test_sequence, predictions, metrics



In [ ]:
if __name__=="__main__":
  np.random.seed(42)
  tf.random.set_seed(42)

  model, test_sequence, predictions, metrics=main()

